In [2]:
%reload_ext tensorboard
%reload_ext autoreload

# Normalising Soft Ambient Flows

In [3]:
import os
import time
import math
import torch
import numpy as np
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
import torch.distributions as tdist
import torchdyn.nn.node_layers as tdnl

from joblib import dump, load
from sklearn.decomposition import PCA
from pdmtut.core import GenerativeModel
from pytorch_lightning import loggers as pl_loggers
from regilib.core.distributions import MultivariateNormal
from regilib.core.dynamics.dynamics import RegularisedDynamics
from regilib.core.dynamics.dynamical_state import DynamicalState
from regilib.core.invertible_modules import NormalisingFlow
from regilib.core.invertible_modules.bijective import ContinuousAmbientFlow

In [4]:
store_results = True
load_models = True

## Introduction

## Implementation

In [5]:
class NormalisingSoftAmbientFlows(NormalisingFlow, pl.LightningModule, GenerativeModel):
    class FunctionDynamics(nn.Module):
        def __init__(self):
            super().__init__()
            
            self._in_channels = 3
            self._out_channels = 3
            
            # expected format: N x (C * L)
             # +1 for time, + 1 for noise condition
            self.fc1 = nn.Linear(self._in_channels + 2, 128)
            self.fc2 = nn.Linear(128, 256)
            self.fc3 = nn.Linear(256, 256)
            self.fc4 = nn.Linear(256, 128)
            self.fc5 = nn.Linear(128, self._out_channels)  
            
        @property
        def in_channels(self):
            return self._in_channels

        @property
        def out_channels(self):
            return self._out_channels

        def forward(self, ds):
            x = torch.cat([ds.state, ds.condition, ds.t], -1)
            x = F.tanh(self.fc1(x))
            x = F.tanh(self.fc2(x))
            x = F.tanh(self.fc3(x))
            x = F.tanh(self.fc4(x))
            x = self.fc5(x)
            return x
    
    def __init__(self, input_dimensions = 3):
        super().__init__(
            base_distribution=MultivariateNormal(torch.zeros(3), torch.eye(3)))
        
        # [a,b] interval of standard deviation of noise distribution
        self.a, self.b = 0, 0.1
        
        self.input_dimensions = input_dimensions
        
        # state=[l, e, n | state]
        self.aug1 = tdnl.Augmenter(augment_dims=3)
        self.af1 = ContinuousAmbientFlow(
            dynamics=RegularisedDynamics(fdyn=NormalisingSoftAmbientFlows.FunctionDynamics()),
            sensitivity='autograd', default_n_steps=5
        )
        
    def noise_enhance_data(self, x, c=None, seed=None, perturb_state=True):
        ds = x.clone() if isinstance(x, DynamicalState) else DynamicalState(state=x)

        n_samples, device = ds.state.shape[0], ds.state.device
        
        # sample uniform distribution c ∈ [a, b]
        if c is None:
            if seed is not None: torch.manual_seed(seed)
            c = torch.FloatTensor(n_samples).uniform_(self.a, self.b)[:,None].to(device)
        else:
            c = c*torch.ones(n_samples, 1)

        if perturb_state: 
            # sample gaussian noise ν ∈ 𝓝(0, I*c)
            if seed is not None: torch.manual_seed(seed)
            nu = torch.randn(n_samples, 3, device=device) * c
            
            ds.state = x + nu # perturb datapoint

        # scale c so that it matches [-1,+1] interval of points
        scale_c = 2*((c - self.a) / (1.e-10 + (self.b - self.a))) - 1
        ds.condition = scale_c # store std as condition
        
        return ds
 
    # Region NormalisingFlow
    def forward(self, x, af_estimate=True):
        ds = x.clone() if isinstance(x, DynamicalState) else DynamicalState(state=x)
        
        ds = super().forward(ds)
        ds = self.af1.dynamics.update_ds(ds, self.aug1(ds['state']))
        ds = self.af1.forward(ds, estimate_trace=af_estimate)
        return ds

    def inverse(self, x, af_estimate=True):
        ds = x.clone() if isinstance(x, DynamicalState) else DynamicalState(state=x)
        
        ds = self.af1.dynamics.update_ds(ds, self.aug1(ds['state']))
        ds = self.af1.inverse(ds, estimate_trace=af_estimate)
        ds = super().inverse(ds)
        return ds
    
    # Region GenerativeModel
    def encode(self, X, c, seed=3):
        ds = DynamicalState(state=X)
        ds = self.noise_enhance_data(ds, c=c, seed=seed)
        ds = self.inverse(ds)
        return ds['state'].cpu().detach()

    def decode(self, z, c, seed=3):
        #TODO: fix
        breakpoint()
        ds = DynamicalState(state=z)
        ds = self.noise_enhance_data(ds, c=c, seed = 1)
        ds = self.forward(ds)
        return ds['state'].cpu().detach()
    
    def save(self, path):
        torch.save(self, os.path.join(path, 'model.pt'))
    
    def load(path):
        return torch.load(os.path.join(path, 'model.pt'))
            
    def save_exists(path):
        return (
            os.path.isfile(os.path.join(path, 'model.pt')))

    def log_likelihood(self, x, c, seed=3):
        ds = DynamicalState(state=X)
        ds = self.noise_enhance_data(ds, c=c, seed=seed)
        ds = self.inverse(ds, af_estimate=False)
        return ds.log_prob.cpu().detach()

    def sample_posterior(self, n_samples, steps=100, log_prob=False):
        samples = self.sample_prior(n_samples)
       
        return z.detach().cpu()
    
    def fit_model(self, X, X_val=None, path=None):
        start_time = time.time()
        
        if path is None:
            tb_logger = False
            checkpoint_callback=False
        else:
            tb_logger = pl_loggers.TensorBoardLogger(path, version=0)
            checkpoint_callback=True
        
        trainer = pl.Trainer(
            max_epochs=2000, gpus=1, logger=tb_logger,
            checkpoint_callback=checkpoint_callback
        )
        trainer.fit(
            self, train_dataloaders=X, val_dataloaders=X_val)
        elapsed_time = time.time() - start_time
        
        if path is not None: 
            with open(os.path.join(path, 'training_time.txt'), 'w') as f:
                f.write(str(elapsed_time))

    def training_step(self, batch, batch_idx):
        x = batch[0]
        lambda_e, lambda_n = 0.01, 0.01
        
        # state=[x+nu|c]
        ds_x_prime = self.noise_enhance_data(x)
        
        # logp(z_t1) = logp(z_t0) - \int_0^1 - Tr ∂f/∂z(t)
        ds_z = self.inverse(ds_x_prime, af_estimate=True)

        # minimise negative log likelihood and energy
        loss = (-ds_z.log_prob + lambda_e * ds_z.e[:, 0] + lambda_n * ds_z.n[:, 0]
        ).sum() / (x.shape[0]*x.shape[1])

        self.log('train_loss', loss)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        x = batch[0]
        lambda_e, lambda_n = 0.01, 0.01
        
        # state=[x+nu|c]
        ds_x_prime = self.noise_enhance_data(x)
        
        # logp(z_t1) = logp(z_t0) - \int_0^1 - Tr ∂f/∂z(t)
        ds_z = self.inverse(ds_x_prime, af_estimate=True)

        loss = (-ds_z.log_prob + lambda_e * ds_z.e[:, 0] + lambda_n * ds_z.n[:, 0]).sum(
        ) / (x.shape[0]*x.shape[1])

        self.log('validation_loss', loss)
        return {'val_loss': loss}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-2)
        return {
            'optimizer': optimizer,
            'lr_scheduler':
            torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, min_lr=1e-8, factor=0.5, verbose=True,
                patience=100
            ), 'monitor': 'train_loss'
        }

    def __str__(self):
        return 'naf'


## Experiment 1: swiss roll

In [6]:
import pyvista as pv
from pdmtut.datasets import SwissRoll

In [7]:
pv.set_plot_theme("document")

model_save_path = '../results/swiss_roll/snf'

if store_results:
    result_save_path = '../results/swiss_roll/snf'
    pv.set_jupyter_backend('None')
else:
    pv.set_jupyter_backend('ipygany')
    result_save_path = None

In [ ]:
dataset = SwissRoll(n_samples=100**2, seed=11)

if load_models and NormalisingSoftAmbientFlows.save_exists(model_save_path):
    model = NormalisingSoftAmbientFlows.load(model_save_path)
else:
    model = NormalisingSoftAmbientFlows()
    model.fit_model(
        X=dataset.train_loader(batch_size=512),
        X_val=dataset.validation_loader(batch_size=512),
        path=result_save_path)
    
    if store_results:
        model.save(model_save_path)
        
model = model.eval()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type                  | Params
------------------------------------------------------------
0 | base_distribution | MultivariateNormal    | 0     
1 | aug1              | Augmenter             | 0     
2 | af1               | ContinuousAmbientFlow | 132 K 
------------------------------------------------------------
132 K     Trainable params
0         Non-trainable params
132 K     Total params
0.531     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/bawaw/.conda/envs/pdm_tutorial/lib/python3.8/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/home/bawaw/.conda/envs/pdm_tutorial/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:326: UserWarning: The number of training samples (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [8]:
%tensorboard --logdir ../results/swiss_roll/snf

### Input Representation

In [9]:
from pdmtut.vis import plot_representation

In [14]:
z = model.encode(dataset.X)
z_extremes = model.encode(dataset.y_extremes)

TypeError: encode() missing 1 required positional argument: 'c'

In [11]:
plot_representation(z.numpy(), index_colors=dataset.index_colors, z_extremes=z_extremes, interpolate_background=True, root=result_save_path)

NameError: name 'z' is not defined

### Input Reconstruction

In [15]:
from pdmtut.vis import plot_reconstruction

In [16]:
z = model.encode(dataset.X)
x = model.decode(z)

TypeError: encode() missing 1 required positional argument: 'c'

In [17]:
plot_reconstruction(dataset.unnormalise_scale(x).numpy(), dataset.index_colors, root=result_save_path)

NameError: name 'x' is not defined

### Density Estimation

In [18]:
from pdmtut.vis import plot_density

In [19]:
log_likelihood = model.log_likelihood(dataset.X)

TypeError: log_likelihood() missing 1 required positional argument: 'c'

In [20]:
plot_density(dataset.unnormalise_scale(dataset.X).numpy(), log_likelihood.numpy(), root=result_save_path)

NameError: name 'log_likelihood' is not defined

### Generate Samples

In [21]:
from pdmtut.vis import plot_generated_samples

In [22]:
generated_samples = model.sample_posterior(100**2)
generated_samples_log_likelihood = model.log_likelihood(generated_samples)

NameError: name 'z' is not defined

In [23]:
plot_generated_samples(dataset.unnormalise_scale(generated_samples).numpy(), generated_samples_log_likelihood.numpy(), root=result_save_path)

NameError: name 'generated_samples' is not defined

### Interpolation

In [24]:
from pdmtut.vis import plot_interpolation
from scipy.interpolate import interp1d

In [25]:
z_extremes = model.encode(dataset.y_extremes)
uniform_state, uniform_log_prob, _ = dataset.sample_points_uniformly(n_samples=100**2, seed=11)

linfit1 = interp1d([1,20], z_extremes[:2].numpy(), axis=0)
linfit2 = interp1d([1,20], z_extremes[2:].numpy(), axis=0)

interpolated_points_1 = model.decode(torch.Tensor(linfit1(np.arange(1,21))))
interpolated_points_2 = model.decode(torch.Tensor(linfit2(np.arange(1,21))))

TypeError: encode() missing 1 required positional argument: 'c'

In [26]:
plot_interpolation(
    dataset.unnormalise_scale(interpolated_points_1).numpy(), 
    dataset.unnormalise_scale(interpolated_points_2).numpy(), 
    uniform_state.detach().view(100, 100, 3).permute(2, 0, 1).numpy(),
    uniform_log_prob.numpy(), root=result_save_path
)

NameError: name 'interpolated_points_1' is not defined